In [1]:
import pandas as pd
from kafka import KafkaConsumer, KafkaProducer
from time import sleep
from json import dumps
import json
import pyodbc
from decimal import Decimal
from datetime import date, datetime

In [2]:
def extract_from_sql_server():
    
    server = 'LAPTOP-OHGJHODB\\SQLEXPRESS'
    database = 'EmployeeDB'
    
    connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                                'SERVER=' + server + ';'
                                'DATABASE=' + database + ';'
                                'Trusted_Connection=yes;')
    
    cursor = connection.cursor()
    cursor.execute("SELECT * FROM employees")
    rows = cursor.fetchall()
    
    columns = [column[0] for column in cursor.description]
    data = [dict(zip(columns, row)) for row in rows]
    
    cursor.close()
    connection.close()
    
    return data

In [3]:
class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Decimal):
            return float(obj)
        if isinstance(obj, (date, datetime)):
            return obj.isoformat()
        return super(CustomEncoder, self).default(obj)

"""def get_last_polled_time():
    try:
        with open('last_polled_time.txt', 'r') as file:
            return file.read().strip()
    except FileNotFoundError:
        return '1900-01-01 00:00:00'

def update_last_polled_time(current_time):
    with open('last_polled_time.txt', 'w') as file:
        file.write(current_time)
"""

"def get_last_polled_time():\n    try:\n        with open('last_polled_time.txt', 'r') as file:\n            return file.read().strip()\n    except FileNotFoundError:\n        return '1900-01-01 00:00:00'\n\ndef update_last_polled_time(current_time):\n    with open('last_polled_time.txt', 'w') as file:\n        file.write(current_time)\n"

In [4]:
def produce_data_to_kafka(data):
    producer = KafkaProducer(
        bootstrap_servers=['18.191.110.113:9092'],
        value_serializer=lambda x: json.dumps(x,cls=CustomEncoder).encode('utf-8'))
    
    
    for record in data:
        producer.send('demo_testing2', value=record)
    
    producer.flush()

In [28]:
data = extract_from_sql_server()
produce_data_to_kafka(data)

In [6]:
df = pd.DataFrame(data)

In [7]:
df.head()

,id,name,position,salary,hire_date,department,email,phone_number,address
0,1,Name1,UX Designer,59027.04,2021-08-16,HR,Name1@example.com,555-123-8651,"8651 Main St, City51, State1"
1,2,Name2,Software Engineer,50915.06,2024-04-01,IT,Name2@example.com,555-123-0301,"301 Elm St, City1, State1"
2,3,Name3,UX Designer,88427.92,2020-12-30,Finance,Name3@example.com,555-123-0208,"208 Elm St, City8, State8"
3,4,Name4,Project Manager,90161.44,2018-04-30,Finance,Name4@example.com,555-123-5946,"5946 Main St, City46, State46"
4,5,Name5,Project Manager,97306.50,2017-01-12,Finance,Name5@example.com,555-123-3843,"3843 Elm St, City43, State43"


In [8]:
df.count


<bound method DataFrame.count of    id    name           position    salary   hire_date department   
0   1   Name1        UX Designer  59027.04  2021-08-16         HR  \
1   2   Name2  Software Engineer  50915.06  2024-04-01         IT   
2   3   Name3        UX Designer  88427.92  2020-12-30    Finance   
3   4   Name4    Project Manager  90161.44  2018-04-30    Finance   
4   5   Name5    Project Manager  97306.50  2017-01-12    Finance   
5   6   Name6    Project Manager  55310.52  2019-03-14  Marketing   
6   7   Name7     Data Scientist  77702.33  2020-11-18  Marketing   
7   8   Name8  Software Engineer  67685.71  2016-10-11         HR   
8   9   Name9     Data Scientist  93440.71  2022-02-08    Finance   
9  10  Name10  Software Engineer  90446.50  2019-02-11         HR   

                email  phone_number                        address  
0   Name1@example.com  555-123-8651   8651 Main St, City51, State1  
1   Name2@example.com  555-123-0301      301 Elm St, City1, State1  
